In [5]:
import os
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

In [6]:
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
#!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [787 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelea

In [7]:
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

In [52]:
# Start a SparkSession
import findspark
findspark.init()

In [53]:
!wget https://jdbc.postgresql.org/download/postgresql-42.3.6.jar

--2022-05-26 22:28:43--  https://jdbc.postgresql.org/download/postgresql-42.3.6.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1041072 (1017K) [application/java-archive]
Saving to: ‘postgresql-42.3.6.jar.1’

postgresql-42.3.6.j 100%[===================>]   1017K  --.-KB/s    in 0.1s    

2022-05-26 22:28:43 (6.67 MB/s) - ‘postgresql-42.3.6.jar.1’ saved [1041072/1041072]



In [56]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.3.6.jar").getOrCreate()

In [58]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Tools_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
tools_data = spark.read.csv(SparkFiles.get("amazon_reviews_us_Tools_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
tools_data.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   15785389|R2UM5QMHBHC90Q|B00H5U9ZD6|     115362950|WallPeg 12 sq ft ...|           Tools|          5|            0|          0|   N|                Y|Great organizer, ...|Very nice. Will o...| 2015-08-31|
|         US|   47910848|  RF0D1LEIF6L7|B001TJGCS0|     570955425|Nite Ize Nite Daw...|           Tools|          4|    

In [59]:
#Count the number of records (rows) in the dataset.
print(tools_data.count())

1741100


In [60]:
#**Transform** the dataset to fit the tables in the [schema file](../Resources/schema.sql). Be sure the DataFrames match in data type and in column name.
tools_data.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [61]:
#create dataframe for CREATE TABLE review_id_table (
#  review_id TEXT PRIMARY KEY NOT NULL,
#  customer_id INTEGER,
#  product_id TEXT,
#  product_parent INTEGER,
#  review_date DATE -- this should be in the formate yyyy-mm-dd
#);

review_id_table_data = tools_data.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_table_data.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R2UM5QMHBHC90Q|   15785389|B00H5U9ZD6|     115362950| 2015-08-31|
|  RF0D1LEIF6L7|   47910848|B001TJGCS0|     570955425| 2015-08-31|
|  RM6YKIWQVNSY|   36328996|B000NIK8JW|     128843593| 2015-08-31|
|R1RL3L68ASPS36|   51785809|B008ZYRGUA|     407828107| 2015-08-31|
|R1U4XFBFAG34CY|   40757491|B00K5CA0GC|     490746675| 2015-08-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [62]:
#-- This table will contain only unique values
#CREATE TABLE products (
#  product_id TEXT PRIMARY KEY NOT NULL UNIQUE,
#  product_title TEXT
#);
products_data = tools_data.select(["product_id", "product_title"])
products_data.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00H5U9ZD6|WallPeg 12 sq ft ...|
|B001TJGCS0|Nite Ize Nite Daw...|
|B000NIK8JW|Stanley 84-058 4 ...|
|B008ZYRGUA|Powerextra 14.4V ...|
|B00K5CA0GC|Waterproof Invisi...|
+----------+--------------------+
only showing top 5 rows



In [63]:
#-- Customer table for first data set
#CREATE TABLE customers (
#  customer_id INT PRIMARY KEY NOT NULL UNIQUE,
#  customer_count INT
#);

customer_count = tools_data.select(["customer_id"]).count()
customer_count


1741100

In [64]:
customers_data = tools_data.groupBy('customer_id').count()
customers_data_counts = customers_data.withColumnRenamed('count', 'customers_count')
customers_data_counts.show()

+-----------+---------------+
|customer_id|customers_count|
+-----------+---------------+
|   45978717|              2|
|   43622307|              1|
|     740678|              4|
|   29045703|              1|
|   52484548|              3|
|   42560427|              4|
|   17067926|              1|
|   10093406|              1|
|   44979559|              1|
|   19432125|              1|
|   26079415|              1|
|   29931671|              1|
|   12945150|              3|
|   12036434|              1|
|   14230926|              1|
|   45015535|              1|
|   39064792|              1|
|   20709090|              1|
|   45074906|              1|
|   26767269|              2|
+-----------+---------------+
only showing top 20 rows



In [65]:
#-- vine table
#CREATE TABLE vine_table (
#  review_id TEXT PRIMARY KEY,
#  star_rating INTEGER,
#  helpful_votes INTEGER,
#  total_votes INTEGER,
#  vine TEXT
#);

vine_data = tools_data.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_data.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R2UM5QMHBHC90Q|          5|            0|          0|   N|
|  RF0D1LEIF6L7|          4|            0|          0|   N|
|  RM6YKIWQVNSY|          1|            6|          6|   N|
|R1RL3L68ASPS36|          4|            0|          0|   N|
|R1U4XFBFAG34CY|          5|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [67]:
 #* **Load** the DataFrames that correspond to tables into an RDS instance. **Note:** This process can take up to 10 minutes for each. Be sure that everything is correct before uploading.
mode="append"
jdbc_url = "jdbc:postgresql://database-1.cloe6i70wwpd.us-east-1.rds.amazonaws.com:5432/PR1"
#jdbc_url = "jdbc:postgresql://192.168.86.77:5432/productreview"
config = {"user":"postgres",
          "password": "postgres",
          "driver":"org.postgresql.Driver"}

In [68]:
# Write review_id_table_df table RDS

review_id_table_data.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [74]:
# Write products_data  to RDS
products_data = products_data.dropDuplicates()
products_data.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [76]:
# Write DataFrame to customers table in RDS

customers_data_counts.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [78]:
# Write vine_data to customers table in RDS

vine_data.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)